# 모델 학습과 예측

이제 데이터 분석을 마치고 실제 학습 단계로 들어가 보겠습니다. 이 섹션에서는 다소 복잡해 보이는 코드들이 등장하는데요, 처음 보시는 분들은 당황하실 수 있습니다. 하지만 걱정하지 마세요! 지금 당장 이 코드들을 모두 이해할 필요는 없습니다.



## 학습 과정 개요

이번 섹션에서는 크게 다음과 같은 작업을 수행합니다:

1. 학습에 사용할 변수 선택
   - 'Pclass': 좌석 등급
   - 'Age': 나이
   - 'SibSp': 동승한 형제/배우자 수
   - 'Parch': 동승한 부모/자녀 수
   - 'Fare': 요금

2. 데이터 분할 (8:2 비율)
   - 학습용 데이터 (80%)
   - 검증용 데이터 (20%)

3. 모델 학습

4. 생존 여부 예측

   


:::{admonition} More Info
**왜 이 섹션이 필요한가요?**

이 섹션의 내용이 기술적으로 복잡해 보일 수 있지만, 실제 예측 결과를 얻기 위해서는 반드시 필요한 과정입니다. 마치 맛있는 요리를 만들기 위해 복잡한 재료 손질 과정을 건너뛸 수 없는 것처럼요.

지금은 이 코드들이 생소하고 어렵게 느껴지시더라도 걱정하지 마세요. 이것은 아주 자연스러운 것입니다. 지금은 "아, 이런 과정을 통해 컴퓨터가 학습을 하는구나" 정도로 이해하시면 충분합니다.

책의 뒷부분에서 이러한 코드들이 어떤 의미인지, 각각의 설정값들은 무엇을 의미하는지 자세히 다룰 예정입니다. 그때가 되면 지금은 복잡해 보이는 코드들이 훨씬 이해하기 쉬워질 것입니다.

차근차근 앞으로 나아가면서 하나씩 이해해 나가시면 됩니다. 지금 당장 모든 것을 이해하려고 하지 마시고, 이런 과정이 있다는 것만 기억해 두세요!
:::

:::{note}
**사전 준비**

아래에 있는 **준비 코드**는 이번 섹션의 학습을 위해 필요한 이전 섹션의 코드입니다. 새로운 내용은 없으니 그대로 실행해주시면 됩니다.

이는 마치 요리 책에서 '이전 단계에서 준비해둔 반죽을 사용하여...'와 같은 설명과 비슷한 역할을 합니다. 학습의 연속성을 위해 필요한 과정이니 부담 없이 진행해주세요.
:::

💻 **준비 코드**

In [1]:
import pandas as pd

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./gender_submission.csv')

## 학습에 사용할 변수 선택

먼저 생존 여부를 예측하기 위해 어떤 정보들을 사용할지 선택해야 합니다. 이번 섹션에서는 5가지 숫자형(Numeric) 피쳐만 사용하겠습니다. 문자형(Object) 피쳐는 추가적인 전처리 과정이 필요하기 때문에 일단 제외하고, 아래 5개의 숫자형 피쳐만 사용하여 첫 번째 모델을 만들어보겠습니다.

- Pclass: 좌석 등급 (1등석, 2등석, 3등석)
- Age: 승객의 나이
- SibSp: 함께 탑승한 형제자매, 배우자의 수
- Parch: 함께 탑승한 부모, 자녀의 수
- Fare: 티켓 요금



📝 **프롬프트**

```
'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'를 inc_fts 변수에 저장해 줘
```


💻 **코드 & 실행결과**

In [2]:
inc_fts = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

이 숫자형 피쳐들은 승객의 생존과 관련이 있을 것으로 예상되어 추가했습니다. 예를 들어, 좌석 등급은 구명보트 탑승 우선순위와 관련이 있을 수 있고, 나이나 가족 관계는 비상 상황에서의 대처 능력이나 도움을 받을 수 있는 가능성과 연관될 수 있습니다.



그 다음 코드들은 선택한 이 정보들을 컴퓨터가 학습할 수 있는 형태로 준비하는 과정입니다. 지금 단계에서는 이 부분을 자세히 이해할 필요는 없습니다. 이런 형태로 데이터를 준비한다는 것만 알아두시면 됩니다.

📝 **프롬프트**

```
- inc_fts에 저장한 피쳐들을 대상으로 train 데이터를 X와 y로 분리해줘
- test 데이터는 X_test로 저장해줘
- X, y, X_test의 행과 열을 출력해줘
```


💻 **코드 & 실행결과**

In [3]:
X = train[inc_fts]     # 선택한 정보들
y = train['Survived']  # 실제 생존 여부
X_test = test[inc_fts] # 예측해야 할 데이터의 정보들

print(X.shape, y.shape, X_test.shape) # 행과 열 출력 

(891, 5) (891,) (418, 5)


## 데이터 분할

이제 데이터를 두 그룹으로 나누어 보겠습니다. 전체 데이터의 80%는 학습용으로, 20%는 검증용으로 사용할 것입니다.

이렇게 데이터를 나누는 이유는 무엇일까요? 간단한 비유를 들어보겠습니다. 영어 시험을 준비한다고 생각해봅시다. 여러분은 문제집의 모든 문제를 외워버리는 대신, 일부 문제로 공부를 하고 나머지 문제로 자신의 실력을 테스트해볼 것입니다. 이렇게 해야 실제 시험에서도 좋은 성적을 낼 수 있겠죠?

머신러닝도 마찬가지입니다:

- 학습용 데이터(80%): 컴퓨터가 패턴을 학습하는데 사용
- 검증용 데이터(20%): 학습한 내용을 테스트해보는데 사용



📝 **프롬프트**

```
X와 y를 8:2 비율로 나누어서 X_train, X_valid, y_train, y_valid로 저장해줘
```


💻 **코드 & 실행결과**

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_valid.shape

((712, 5), (179, 5))

이 코드는 데이터를 8:2로 나누는 작업을 수행합니다. 코드의 세부적인 내용은 지금 단계에서는 이해하실 필요가 없습니다. 데이터를 학습용과 검증용으로 나누어 준다는 점만 기억해 주세요.



## 모델 학습

이제 본격적으로 컴퓨터에게 데이터를 학습시켜 보겠습니다. 여기서는 'Random Forest'라는 머신러닝 알고리즘을 사용할 것입니다.

Random Forest는 여러 개의 의사결정 나무를 만들어 이들의 '투표' 결과로 최종 예측을 하는 방법입니다. 마치 어려운 결정을 해야 할 때 여러 전문가의 의견을 종합하는 것과 비슷하다고 생각하시면 됩니다.

📝 **프롬프트**

```
Random Forest 모델로 X_train과 y_train을 학습시켜줘
```


💻 **코드 & 실행결과**

In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

이 코드는 Random Forest 모델을 생성하고 학습하는 과정입니다. `random_state=42`는 실험을 반복해도 같은 결과가 나오도록 하는 설정이며, 꼭 42가 아니어도 됩니다.

지금 단계에서는 이 코드의 세부적인 내용을 이해하실 필요는 없습니다. Random Forest라는 방법으로 컴퓨터가 데이터를 학습한다는 것만 기억해 주세요. 이후에 Random Forest를 포함한 여러 머신러닝 알고리즘에 대해 더 자세히 다루게 될 것입니다.



이제 학습된 모델이 실제로 얼마나 잘 예측하는지 테스트해볼 차례입니다. 앞서 우리는 전체 데이터의 20%를 검증용으로 따로 보관해두었습니다. 이 데이터들의 생존 여부를 모델이 예측하도록 하는 것이죠. 이것은 마치 학생이 모의고사를 보는 것과 비슷합니다. 공부한 내용을 바탕으로 처음 보는 문제를 풀어보는 거죠.

📝 **프롬프트**

```
X_valid를 예측해서 y_pred에 저장해줘
```


💻 **코드 & 실행결과**

In [6]:
y_pred = model.predict(X_valid)
y_pred

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1], dtype=int64)

`y_pred`에는 모델이 예측한 결과가 저장됩니다. 각 승객에 대해 0(사망) 또는 1(생존)의 값이 저장되어 있습니다. 이제 이 예측 결과가 실제 결과와 얼마나 일치하는지 확인해볼 차례입니다.



📝 **프롬프트**

```
- y_valid와 y_pred를 비교해 accuracy를 계산하고, 소수점 다섯째 자리까지 출력해줘
- Validation Score라는 문구를 포함시켜줘
```


💻 **코드 & 실행결과**

In [7]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_valid, y_pred)
print(f"Validation Score: {accuracy:.5f}")

Validation Score: 0.73184


이 코드는 모델의 정확도를 계산합니다. 정확도는 전체 예측 중 맞은 예측의 비율을 의미합니다.

우리 모델의 정확도는 73.184%로 나왔습니다. 이는 모델이 100명의 승객 중 약 73명의 생존 여부를 올바르게 예측했다는 의미입니다. 이때 accuracy_score 함수는 실제 값(y_valid)과 예측 값(y_pred)을 비교하여 이 비율을 계산해줍니다.

정확도는 0에서 1 사이의 값을 가지며, 1에 가까울수록 예측이 정확하다는 뜻입니다:

- 1.0: 완벽한 예측 (100% 정확)
- 0.0: 완전히 틀린 예측 (0% 정확)

지금은 정확도라는 간단한 평가 지표만 사용했지만, 나중에는 다양한 평가 방법에 대해 배우게 될 것입니다.



## 생존 여부 예측

이제 진짜 예측을 해볼 차례입니다. 앞서 검증용 데이터로 모델의 성능을 확인해봤다면, 이번에는 실제 테스트 데이터에 대해 예측을 수행합니다.

📝 **프롬프트**

```
X_test에 대해 예측해서 y_test_pred에 저장해줘
```


💻 **코드 & 실행결과**

In [8]:
y_test_pred = model.predict(X_test)
y_test_pred

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,

이전에 했던 검증용 데이터(X_valid)에 대한 예측과 이번 테스트 데이터(X_test)에 대한 예측은 의미가 조금 다릅니다:

- 검증용 데이터: 실제 답을 알고 있는 데이터로, 모델의 성능을 평가하기 위해 사용
- 테스트 데이터: 실제 답을 모르는 데이터로, 캐글에 제출할 최종 예측값을 얻기 위해 사용

`y_test_pred`에는 각 승객의 생존 여부 예측값(0: 사망, 1: 생존)이 저장됩니다. 이 결과를 캐글에 제출하기 위한 형식으로 만드는 작업이 다음 단계가 되겠네요.



이제 마지막 단계입니다. 예측 결과를 캐글에 제출할 수 있는 형태의 파일로 만들어보겠습니다.

📝 **프롬프트**

```
- submission 데이터의 'Survived' 컬럼에 y_test_pred 값을 저장해줘
- titanic_pred.csv 파일에 저장하고, 첫 5줄을 출력해줘
```


💻 **코드 & 실행결과**

In [9]:
submission['Survived'] = y_test_pred
submission.to_csv('titanic_pred.csv', index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


이 코드는 두 가지 작업을 수행합니다:

1. 우리가 예측한 생존 여부(y_test_pred)를 submission 데이터프레임의 'Survived' 열에 저장합니다.
2. 이 결과를 'titanic_pred.csv'라는 이름의 파일로 저장합니다.

생성된 파일은 승객 ID(PassengerId)와 예측된 생존 여부(Survived)를 포함하고 있습니다.  이 파일을 캐글에 제출한 결과, 0.63397의 점수를 받았습니다. 이는 테스트 데이터 418명 중 약 265명(418 × 0.63397 ≈ 265)의 생존 여부를 정확하게 예측했다는 의미입니다.

```{figure} ./images/13-1.png
---
width: 600px
---
결과파일 제출
```

이 점수는 이전 섹션에서 봤던 성별 기반 예측(gender_submission)의 점수보다는 낮습니다. 그 이유는 현재 모델이 숫자형(Numeric) 변수들만을 사용했기 때문입니다. 성별(Sex)과 같은 중요한 정보들이 문자형(Object) 변수라서 아직 활용하지 못했죠.

하지만 걱정하지 마세요! 이것은 우리의 첫 번째 시도일 뿐입니다. 앞으로 다음과 같은 방법들을 통해 점수를 높여갈 것입니다:

1. 문자형 변수들을 활용하는 방법 배우기
2. 결측치를 더 효과적으로 처리하기
3. 변수들간의 관계를 활용하기
4. 다양한 머신러닝 알고리즘 시도해보기

이렇게 하나씩 개선해가면서, 모델의 성능이 어떻게 향상되는지 함께 확인해보도록 하겠습니다.